## importing libraries

In [1]:

import os 
import pandas as pd
import numpy as np
import spacy
import nltk
import re
import pyarabic.araby as araby
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
#!pip install scispacy
from scispacy.abbreviation import AbbreviationDetector
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from collections import Counter
#!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import TreebankWordTokenizer 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import gensim
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
import nltk
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy.cluster import hierarchy
from abbreviations import schwartz_hearst
import qalsadi.analex as qa
import pyarabic.araby as araby
import naftawayh.wordtag
from nltk.corpus import wordnet
from textblob import TextBlob
import statistics as st

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
C:\Users\lenovo\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## reading in the input 
## takes string input

In [2]:
def read_input_ar(text):
    
    title = []
    keywords =[]

    lang_Dict = []
    for line in araby.sentence_tokenize(text):
        for word in line.split(' '):
            if word not in lang_Dict:
                lang_Dict.append(word)
    
    iter_dict = lang_Dict
    lang_Dict = []
    for i in iter_dict:
        if '\n' in i:
            lang_Dict = lang_Dict + i.split("\n")
        else:
            lang_Dict.append(i)
            
    lang_Dict = [word.replace(".","") for word in lang_Dict]
    keyvalue = dict() 

    i = 0
    for index, line in enumerate(araby.sentence_tokenize(text)):
        new_t = []
        final = []
        if "\n" in line:
            line = line.split('\n' )
            new_t = new_t + line
        else:
            new_t.append(line)
        for m in new_t:
            if "." in m:
                l = m.split('.' )
                final = final + l
            else:
                final.append(m)
        for v in final:
            if (v.strip() != '') and (v.strip() != " "):
                keyvalue[i] = str(v)
                i = i + 1

    key_value_list =[]
    for item in keyvalue.items():
        key_value_list.append(item)
    return title,keywords,keyvalue, key_value_list, lang_Dict

## functions for preprocessing

In [3]:
#this function rplaces the appreviation in the text 
#for example  Natural language processing --> NLP
def replace_acronymss(keyvalue):
        sent_dict = dict() 
        whole_text = ""
        for key, text in keyvalue.items():
            whole_text = whole_text + "#*# " + text
        whole_text = whole_text.lower()
        pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=whole_text, most_common_definition=True)
        for key,value in pairs.items():
            whole_text = whole_text.replace(value, key)
        output_text = whole_text.split("#*# ")
        i = 0;
        for l in output_text:
            if l != "":
                sentence = Brackets(l)
                sent_dict[i] = sentence
                i = i+1
        return sent_dict
    
def tokenize_sent(sent_dict):
    keyvalueToken = dict() 
    TorF = True
    for key, value in sent_dict.items():
        
        words = nltk.word_tokenize(value)
        words = [re.sub(r'(\n-)','',word) for word in words]
        words = [re.sub(r'(\n)',' ',word) for word in words]
        new_words = []
        for word in words:
            if " " in word:
                 new_words = new_words + word.split(" ")
            else:
                new_words.append(word)
        words = new_words   
        new_words = []
        for word in words:
            if "-" in word:
                TorF = True
                check = word.split("-")
                for i in check:
                    if i not in lang_Dict:
                        TorF = False
                if TorF:
                    new_words = new_words + check
                else:
                    new_words.append(word)
            else:
                new_words.append(word)

        keyvalueToken[key] = new_words    
    return keyvalueToken

#continue sent reduction
def conjunctions_ar(tokenize_sent_dict):
    new_sent_dict = dict()
    conjunctive_adverbs= ["وفقا لذلك","أيضاً","على أي حال","إلى جانب ذلك","بالتأكيد","وبالتالي","طرداً وعكساً","أخيراً","علاوة على ذلك","بيد أنّ","بالتبعية","فعلاً","إنّ","بدلاً من أن","بطريقة مماثلة","على الرغم من أنّ","ومع ذلك","بغض النظر","بصرف النظر","بعبارة أخرى","بعبارة أخرى يمكن القول بأن","معناه بعبارة أكثر دقة","جل ما في الأمر أنّ","أو على الأصح","بالتحديد","على وجة التحديد","لا يعني بالضرورة"]
    
    for key, text in tokenize_sent_dict.items():
        new_sent = [word for word in text if word.lower() not in conjunctive_adverbs]
        new_sent_dict[key] = new_sent

    return new_sent_dict

#d Sentence reduction 
def Brackets(text):
    
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub("[\().*?(\)]", "", text)
    
    text=re.sub(r"[\[].*[\]]",r"",text)
    text=re.sub(r'[0-9]+',r'',text)
    text=re.sub(r"[\().*(/)]",r"",text)
    
    
    text=re.sub(r"\s+",r" ",text).strip()
    return text

def AR_POS(Words):
    # using two libraries qalsadi and naftawayh

    nodes_list = ["S#"]
    edges_list = [""]
    tagger = naftawayh.wordtag.WordTagger()
    debug=False;
    limit=500
    nouns = []
    
    for i in range(1,(len(Words))):
        
        analyzer = qa.Analex()
        analyzer.set_debug(debug);
        result = analyzer.check_text(Words[i]);
        T = repr(result[0][0]).split(",")[10].split("=")[1]
        if 'noun' in T.lower():
            nouns.append(Words[i])
            nodes_list.append(Words[i])
            edges_list.append("")
        elif 'verb' in T.lower():
            nouns.append(Words[i])
            nodes_list.append(Words[i])
            edges_list.append("")
        elif 'unknown' in T.lower():
            if tagger.is_noun(Words[i]):
                nouns.append(Words[i])      
        else:
            edges_list.append(Words[i])
            nodes_list.append("")
            
    return nouns , nodes_list , edges_list
#stemmer
def stemming_words(list_of_words):
    stem_list_of_words=[]
    for word in list_of_words:
        if len(word) >= 5:
            if word.startswith("وال") or word.startswith("كال") or word.startswith("بال") or word.startswith("فال") :
                word=word[3:len(word)]
            elif word.startswith("ال"):
                word=word[2:len(word)]
            if word.endswith("تها") or  word.endswith("تهم") or  word.endswith("تكم") or  word.endswith("تنا") :
                word=word[0:len(word)-2]
                word=word.replace("ت","ة")
            elif word.endswith("ها") or  word.endswith("هم") or  word.endswith("كم") or  word.endswith("نا") :
                word=word[0:len(word)-2]
        stem_list_of_words.append(word)
    return stem_list_of_words

#normalization is to remove taskeel, harakat, shadda and hamza 
def arabic_normalization(Text):
    text=araby.strip_harakat(Text)
    text=araby.strip_shadda(text)
    text=araby.strip_tatweel(text)
    text=araby.strip_tashkeel(text)
    text=araby.normalize_hamza(text)
    #text=araby.normalize_ligature(text) #use this in post processing to recap what's normalized
    #remove set of .? and replace it by space then the symbole
    text=re.sub(r"([,.?!])",r" \1",text)
    #remove sequence of white spaces 
    text=re.sub(r"\s+",r" ",text).strip()
    return text

# to get the ngrames from a list of words
def ngramise(sequence):
    bigrams = []
    for bigram in nltk.ngrams(sequence, 2):
        bigrams.append(bigram)
    return bigrams

# removing the hyfen from word an checking if new words exist in the words of the document
#if they exist then the hyfen is removed if not then it is considered one word
def remove_hyfen(wordslist):
    words_new = []
    for word in wordslist:
        digit = False
        lang = False
        if "-" in word:
            words = word.split("-")
            for w in words:
                if w.isdigit():
                    digit = True
            if digit:
                newwords = [w for w in words]
                words_new = words_new + newwords
            else:
                for w in words:
                    if w in lang_Dict:
                        lang = True
                if lang:
                    newwords = [w for w in words]
                    words_new = words_new + newwords
                else:
                    newwords = [word]
                    words_new = words_new + newwords
        else:
            newwords = [word]
            words_new = words_new + newwords
    return words_new
#remove hyphens and tokenizing sentence
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word == "م":
            word = ""
        if word != '':
            new_words.append(word)
    return new_words

# get the frequency of each word in the bigrams
def bigrams_freq(bi_count_list):
    words_freq_bigram = dict()
    bi_count = []
    for list_bi in bi_count_list:
        (w1,w2), i = list_bi
        if w1 in words_freq_bigram.keys():
            words_freq_bigram[w1] = words_freq_bigram[w1] + i
        else:
            words_freq_bigram[w1] = i
        if w2 in words_freq_bigram.keys():
            words_freq_bigram[w2] = words_freq_bigram[w2] + i
        else:
            words_freq_bigram[w2] = i   
    for key,value in words_freq_bigram.items():
        bi_count.append([key,value])
    return bi_count

# some preprocessing steps such as removing stopwords and stemming
def arabic_preprocessing (text,markers_flage=1):
    markers=["في","علي","فوق","تحت","يمين","يسار","و","ثم","او","بل","لكن","حتي","ايضا","قبل","بعد","جيث","ليس","لكن","ولكن","الخ",".",",","?",":",";","/"]
    arabic_stop_word=set(stopwords.words('arabic'))
    arabic_stop_words=list(arabic_stop_word)
    arabic_stop_words.append("جدا")
    arabic_stop_words.append("الخ")
    arabic_stop_words.append("إلخ")
    arabic_stop_words.append("ءلخ")
    list_of_words=word_tokenize(text)
    list_of_stem=stemming_words(list_of_words)
    dict_of_stem={}

    for i in range(len(list_of_stem)):
        dict_of_stem[list_of_stem[i]]=list_of_words[i]

    if markers_flage == 1:
        list_of_stem=[x for x in list_of_stem if x not in arabic_stop_words]
    stem_text=""
    for st in list_of_stem:
        stem_text=stem_text+" "+st   

    return list_of_words,list_of_stem,dict_of_stem
# more preprocessing steps 
def EF_ar(sent_dict):
    final_sent = dict()
    all_words = []
    stopWords = set(stopwords.words('arabic'))
    remove = string.punctuation
    remove = remove.replace("-", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    
    for key, value in sent_dict.items():
        
        value = value.split(" ")
        value = remove_hyfen(value)
        value = remove_punctuation(value)
        filtered_sentence = [w for w in value if w not in stopWords] 
        #replace everything not in the list with space
        filtered_sentence = [re.sub("'", '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub('"', '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub(pattern, '', word) for word in filtered_sentence]
        filtered_sentence = [re.sub(r'[0-9]+', '', word) for word in filtered_sentence]
        filtered_sentence = [word for word in filtered_sentence if len(word.strip()) >1]
        filtered_sentence = [word.strip() for word in filtered_sentence]
 
        final_sent[key] = filtered_sentence
        all_words = all_words + filtered_sentence
    word_frequency = list(Counter(all_words).items())
    return word_frequency, final_sent , all_words


## preprocessing

In [4]:

def preprocess_ar(keyvalue):
    
    sent_dict = replace_acronymss(keyvalue)
    tokenize_sent_dict = tokenize_sent(sent_dict)
    new_sent_dict = conjunctions_ar(tokenize_sent_dict)
    
    sent_list =[]
    for item in sent_dict.items():
        sent_list.append(item)

    all_bigrams = []
    for key, value in sent_dict.items():
        bigrams = ngramise(value)
        all_bigrams = all_bigrams+bigrams

    bi_count = dict(Counter(all_bigrams))
    delete = [key for key,value in bi_count.items() if value < 2]
    # delete the key 
    for key in delete: del bi_count[key]
    bi_count_list =[]
    for key , value in bi_count.items():
        bi_count_list.append([(key),value])

    word_freq ,final_sent, all_words = EF_ar(sent_dict)
    Sentences_list = []
    for item in final_sent.items():
        Sentences_list.append(item)     

    #word frequencies
    #freq of words in title
    title =[]
    title_freq = Counter(title)

    keywords =[]
    Keyword_freq = Counter(keywords)

    w_freq = Counter(all_words)
    final_Word_freq = dict()
    for word in all_words:
        if word not in title_freq.keys():
            title_freq[word] = 0
        if word not in Keyword_freq.keys():
            Keyword_freq[word] = 0
        final_Word_freq[word] = title_freq[word] + Keyword_freq[word] + w_freq[word] 

    ##final list of sentences after preprocessing
    final_input = []
    for (index, sentence) in Sentences_list:
        final_input.append(" ".join(sentence))
        
    keyword_word_freq = []
    for word in all_words:
        keyword_word_freq.append([word,Keyword_freq[word]])  
    title_word_freq = []
    for word in all_words:
        title_word_freq.append([word, title_freq[word]])

    Sentences_list
    List_of_sent = dict()
    for (key, sent) in Sentences_list:
        List_of_sent[key] = " ".join(sent)

    final_Word_freq
    finalwordfreq = []
    for key , value in final_Word_freq.items():
        finalwordfreq.append([key,value])

    return (Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
            ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq)
    

## computation of weights

In [5]:
# takes in a list of nodes and outputs their weights using their frequencies
#weights
def get_weights(list_of_nodes):
    import pandas as ps
    import statistics as st
    node_weight_dict = dict()
    node_list= list_of_nodes
    FWL= finalwordfreq
    TWL= title_word_freq
    KWL= keyword_word_freq
    BWL= bi_count 
    PWL= Pnouns
    

    F=[]     #freq of words  which appeared in nodes WRT document
    T=[]     #freq of words  which appeared in nodes WRT title
    K=[]     #freq of words  which appeared in nodes WRT keyword list 
    B=[]     #freq of words which appeared in nodes WRT Bi_Gram list 
    P=[]     #list of detection the proper noune 
    FW=[]    
    Xt,Xk,Xbi,Xp,Xb=1,1,1,1,1
    weighted_node=[]
    
    
    for i in node_list:
        flag=0
    
        for j in FWL:
            if(i==j[0]):
                f=j[1]
                F.append(f)
                flag=1
                
        if(flag!=1): 
            F.append(0) 

    for i in node_list:
        flag=0
        for j in TWL:
            if(i==j[0]):
                t=j[1]
                T.append(t)
                flag=1
        if(flag!=1): 
             T.append(0)
                
        
    for i in node_list:
        flag=0
        for j in KWL:
            if(i==j[0]):
                k=j[1]
                K.append(k)
                flag=1
        if(flag!=1): 
            K.append(0)
            
            
            
    for i in node_list:
        flag=0
        for j in BWL:
            if(i==j[0]):
                b=j[1]
                B.append(b)
                flag=1
        if(flag!=1): 
            B.append(0)
            
            
    for i in node_list:
        flag=0
        for j in PWL:
            if(i==j):
                P.append(1)
                flag=1
        if(flag!=1): 
            P.append(0)             
    
    for i in FWL:
        fw=i[1]
        FW.append(fw)
    
    if FW == []:
        avrage_word_frequency= 0
        median_word_frequency= 0 
    elif len(FW) != 0:
        avrage_word_frequency=st.mean(FW)
        median_word_frequency=st.median(FW) 
    else:
        avrage_word_frequency= 0
        median_word_frequency= 0 
        
    AW=abs(avrage_word_frequency-median_word_frequency)
    
    for i in range(len(node_list)):
        WN=F[i]+Xt*AW*T[i]+Xk*AW*K[i]+Xb*AW*B[i]+Xp*AW*P[i]
        weighted_node.append(WN)
        node_weight_dict[node_list[i]] = WN
    
    return node_weight_dict,weighted_node
    

## functions for graph algorithm (algorithm 2)

In [6]:
#edge between source and distenation
def get_edge(source,distenation):
    return edgeget[source,distenation]


#distination node list
# gets the weights of the distination nodes given a source node , all node weights and source-destination dictionary 
def getweights_of_dist_nodes(source_destinations,sourcenode,All_nodes_ww):  
    x=source_destinations[sourcenode]
    distenation_node_list={}
    weights=[]
    nothing=0
    for i in range(len(x)):
        if ((x[i])==('E#')) or ( x[i] not in All_nodes_ww):
            #nothing is a flag for destination nodes
            #if one then there is none
            nothing=1
        else:
            distenation_node_list[x[i]]=All_nodes_ww[x[i]]
            weights.append((All_nodes_ww[x[i]]))
        if len(weights) != 0:
            nothing=0
    return distenation_node_list,weights,nothing


## Algorithm 2 get candidate edges

In [7]:

#check for candidate edges between a source and a destination given a criteria 
def get_candidate_edges(source_destinations,node_listx,wxx,out_carteria,All_weight,input_nodes,All_SD):
    c=[]
    xxx = []
    no_list,totalweights=node_listx,All_weight

    source_node_weight=0
    distination_node_weight=0
    average_of_the_node_weight=np.average(totalweights)#(np.sum(node_list)/number_of_nodes)
    median_of_the_node_list_weight=np.median(totalweights)
  
    if average_of_the_node_weight>median_of_the_node_list_weight:#can be replaced with source=max(avg,median)
        source_node_weight=average_of_the_node_weight
    else:
        source_node_weight=median_of_the_node_list_weight

    for node,weight in no_list.items():
        
        if weight>=source_node_weight:#بيعرف اذا كانت بدايه الجمله ولا لا 
            outnodes1,outnodesweights,flagnothing=getweights_of_dist_nodes(source_destinations,node,no_list)#فنكشن جوا الجي بتجبلي كل الديستناشن بتاعت النودواختار بينهم هسقط مين
            outnodes2,nodes2weights,flag2nothing=getweights_of_dist_nodes(All_SD,node,input_nodes)
            outnodes1=[(i,j) for (i,j) in outnodes1.items() if j!=0 ] 
            outnodes2 =[(i,j) for (i,j) in outnodes2.items() if j!=0 ] 
        
            if flagnothing==0:
                #destination nodes exsist
                #get their average and max weights for avg and max criterias
                average_weight=np.average(nodes2weights)
                max_weight=max(nodes2weights)
            else:
                #no destination nodes 
                average_weight=0
                max_weight=0
            # get threshold of destination weights according to criteria 
            if (out_carteria =="avg"):
                distination_node_weight=average_weight
            elif (out_carteria =="node_avg"):
                distination_node_weight=source_node_weight
            elif (out_carteria=="max"):
                distination_node_weight=max_weight
            else:
                distination_node_weight=average_weight


            if (bool(outnodes1)==False):
                # no destination nodes 
                nothing=1  
            else: 
                # check for the candidate edges
                for out,weight_of_out_node in outnodes1:
                    if weight_of_out_node>=distination_node_weight and flagnothing==0:
                        c.append(get_edge(node,out))# append the edge
                        xxx.append([node,out])

    cx=[]
    for i in range(0,len(c)):
        cx.append(c[i][0])
    return cx
  


## functions for building the Graph 

In [8]:

# Add a vertex to the dictionary
def add_vertex(v):
    global graph
    global vertices_no
    if v not in graph:
        vertices_no = vertices_no + 1
        graph[v] = []

# Add an edge between vertex v1 and v2 with edge weight e
#v1 is the source node and v2 is the distenation node
edgeget={}
def add_edge(v1, v2, e_order, e_value):
    global graph
    # Check if vertex v1 is a valid vertex
    if ((v1 in graph) and (v2 in graph)):
        temp = [v2, e_order, e_value]
        edgeget[v1,v2]=[e_value]
        graph[v1].append(temp)

# Print the graph
def print_graph():
    global graph
    for vertex in graph:
        for edges in graph[vertex]:
            print(vertex, " -> ", edges[0], "    edge order:", edges[1], "    edge value:", edges[2])

# get the destinations
def get_destinations(source):
    global graph
    distinations=[]
    if(source in graph):
        for elements in graph.get(source):
            if(elements[0] not in distinations):
                distinations.append(elements[0])
    return distinations

# get the edges
def get_edges(source,dist):
    global graph
    Edge=""
    if(source in graph and dist in graph):
        for elements in graph.get(source):
            if(elements[0]==dist):
                Edge= Edge + elements[2]
    return Edge


## functions for post-processing

In [9]:
# get the weight of the sentence order in the input document
def sentence_order(count_summ, index):
    return(count_summ - index)/count_summ

#given a list of candidate sentences rank them according to ranking criteria
def compute_rank(candidate_summary):
    ranks = {}
    candidate_summary = reorder(candidate_summary)
    
    for i, sent in enumerate(candidate_summary):
        rank = sentence_order(len(candidate_summary), i)
        ranks[sent] = rank
        
    sort_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)    
    return ranks   

#vectorize sentences for clustering
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
     
    return np.asarray(sent_vec) / numw

#numner of words in a sentence
def get_words_count(sent):
    len_sent = 0
    for i in sent.split(" "):
        len_sent = len_sent +1
    return len_sent

# reorder the summary based on the order from the input text
def reorder(summary):
    index_summ = []
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    order = dict()
    final_order = []
    for sent in summary:
        if sent in value_sent:
            index_sent = value_sent.index(sent)
            index_summ.append(index_sent)
            order[index_sent] = sent

    for i in sorted(order):
        final_order.append(order[i])

    return final_order

#concatenate the sentences in the candidate summary
def concat(final_order):
    return(" ".join(final_order))
 
# get the index of a sentence from the input text
def get_index(sent):
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    index_sent = value_sent.index(sent)
    return index_sent  


## Post-Processing and algorithm four

In [10]:
def post_processing_ar(list_of_candidates_summeries):
    candidate_summary = list_of_candidates_summeries
    # rank sentneces
    sentences_ranks = compute_rank(candidate_summary)
    # order them 
    sentences_ordered =  [key for key,value  in sentences_ranks.items()]
    summary_length =0
    visited_sentences = []
    
    sentences = sentences_ordered
    
    arabic_stop_word=set(stopwords.words('arabic'))
    arabic_stop_words=list(arabic_stop_word)
    arabic_stop_words.append("جدا")
    arabic_stop_words.append("الخ")
    arabic_stop_words.append("إلخ")
    arabic_stop_words.append("ءلخ")
    
    tfidfvect = TfidfVectorizer(stop_words=arabic_stop_words)
    X = tfidfvect.fit_transform(sentences)

    if len(sentences) <1: #clusters can't be more than number of words
        n_clusters = len(sentences)
    else:
        n_clusters = 1
    clf = KMeans(n_clusters = n_clusters,init = 'k-means++', max_iter = 1000)
    labels = clf.fit_predict(X)
    clustered_sentences =[[] for i in range(5)]

    cluster_map = pd.DataFrame()
    cluster_map['data_index'] = sentences
    cluster_map['cluster'] = clf.labels_
    #print(cluster_map)
    return n_clusters,cluster_map

# clusting algorithm if clusters = 1 then this is just picking top sentneces
def algorithm_4_ar(summary_length,threshold,max_words_limit,list_of_candidates_summeries,n_clusters,cluster_map):
    candidate_summary = list_of_candidates_summeries
    summary_length = 0
    while (len(visited_sentences) < len(candidate_summary)) and ((summary_length)< max_words_limit) :
        for i in range(n_clusters):
            clustered_sentences = cluster_map[cluster_map.cluster == i]['data_index'].tolist()
            if summary_length >= (max_words_limit+threshold):
                break
            for sent in clustered_sentences:
                if sent not in visited_sentences :
                    temp_length = summary_length + 1
                    if temp_length < max_words_limit :
                        summary_length = temp_length
                        final_summary.append(sent)
                        visited_sentences.append(sent)
                        print("ADD Sentence of index ", get_index(sent))
                        break
                    elif(temp_length >= max_words_limit) and (temp_length <= (max_words_limit + threshold)):
                        summary_length = temp_length
                        final_summary.append(sent)
                        visited_sentences.append(sent)
                        print("ADD Sentence of index ", get_index(sent))
                        break
                    elif temp_length > (max_words_limit + threshold):
                        visited_sentences.append(sent)
                        print("ADD Sentence of index ", get_index(sent))
        
    return final_summary

## iterative part of the code

In [11]:

def iter_graph_ara(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria):
    All_SD = {}
    input_nodes = []
    sent_nodes = {}      
    nodes_list = []
    input_text = candidate_summ
    last_summ_len = candidate_summ_len
    Last_summ = candidate_summ
    sent_idx = can_edges
    #print("sent_idx   ", sent_idx)
    can_edges=[]
    List_of_sent = list_pre_cansumm
    list_pre_cansumm = []
    list_of_candidates_summeries = []
    All_nodes=[]
    nodes_list = []
    # driver code
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    #print(S.values())
    index = 0
    for sent_index, sentence in zip(sent_idx,List_of_sent):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""
        sentence=arabic_normalization(sentence)
        
        Words,list_of_stem,dict_of_stem=arabic_preprocessing(sentence)

        words=stemming_words(Words)

        Types, nodes_list , edges_list = AR_POS(Words)


        list_of_nodes =[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i

        list_of_nodes.remove("S#")    
        add_vertex("E#")
        nodes_list.append("E#")
        edges_list.append("")

        counter =0
        order =0
        for j in nodes_list :
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break
                
        LIST_OF_NODES=[]
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)

        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)
        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)


        input_nodes = input_nodes + list_of_nodes
        sent_nodes[index] = (source_destinations,list_of_nodes)
        
        graph = {}
        source_destinations={}
        index = index + 1

        
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes)
    All_weight = list_of_weights
    
    index = 0
    for sent_index, sentence in zip(sent_idx,List_of_sent): 
        (source_destinations,list_of_nodes) = sent_nodes[index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])


        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        #list_of_candidates_summeries = []
        if len(list_of_can_edges)>=1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[index] and (List_of_sent[index].strip()) ):
                #print("PICKED  ", index, "   ", sent_index)
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[index])
                can_edges.append(sent_index)

        index = index + 1
        
    candidate_summ = reorder(list_of_candidates_summeries)
    candidate_summ_len = 0
    for index ,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + 1

    return Cand_edge,list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges






## Final Function
## Takes in an input text and outputs a summary

In [22]:
def summarization_ara(Text):
    All_SD = {}
    input_nodes = []
    sent_nodes = {}      
    nodes_list = []
    global vertices_no
    vertices_no =0
    
    can_edges = []
    global keyvalue
    global key_value_list
    global lang_Dict

    title,keywords,keyvalue, key_value_list, lang_Dict =  read_input_ar(Text) 
    out_carteria="avg"
    print("Print key value sentence")
    print(keyvalue)
    global Sentences_list
    global final_input
    global all_bigrams
    global bi_count_list
    global word_freq
    global all_words
    global finalwordfreq
    #global List_of_sent
    global title_word_freq
    global keyword_word_freq
    global All_nodes
    global list_of_candidates_summeries
    
    global list_pre_cansumm
    
    (Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
     ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq) = preprocess_ar(keyvalue)
    print("INPUT after Pre-Processing")
    print(final_input)
    global bi_count
    global Cand_edge
    Cand_edge =[]
    Word_Freq = {}
    for key, value in finalwordfreq:
        Word_Freq[key] = value
    

    bi_count = bigrams_freq(bi_count_list) 
    Last_summ = keyvalue.values()
    last_summ_len = 0
    for value in keyvalue.values():
        last_summ_len = last_summ_len + 1


    All_nodes=[]
    list_of_candidates_summeries=[]
    list_pre_cansumm = []

    ##tokanization  
    tokenizer = TreebankWordTokenizer()

    ##lemmatization
    lemmatizer = WordNetLemmatizer() 

    # Add an edge between vertex v1 and v2 with edge weight e
    #v1 is the source node and v2 is the distenation node
    edgeget={}

    
    # driver code
    global graph 
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    
    input_nodes = []
    sent_nodes = {}
    global Pnouns
    Pnouns = []
    
    for sent_index, sentence in enumerate(List_of_sent.values()):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""
        sentence=arabic_normalization(sentence)
        Words,list_of_stem,dict_of_stem=arabic_preprocessing(sentence)
        words=stemming_words(Words)
        Types, nodes_list , edges_list = AR_POS(Words)
        for node in nodes_list:
            if node != "":
                Cand_edge.append(node)

        list_of_nodes =[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i

        list_of_nodes.remove("S#")    
        add_vertex("E#")
        nodes_list.append("E#")
        edges_list.append("")
        
        counter =0
        order =0
        for j in nodes_list :
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break
                
        LIST_OF_NODES=[]
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)


        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)
        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)


        input_nodes = input_nodes + list_of_nodes
        sent_nodes[sent_index] = (source_destinations,list_of_nodes)
        
        graph = {}
        source_destinations={}
        
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes)
    All_weight = list_of_weights
    for sent_index, sentence in enumerate(S.values()): 
        (source_destinations,list_of_nodes) = sent_nodes[sent_index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])
        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        #list_of_candidates_summeries = []
        if len(list_of_can_edges)>= 1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[sent_index] and (List_of_sent[sent_index].strip()) ):
                #print("picked ", sent_index)
                #print(keyvalue[sent_index])
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[sent_index])
                can_edges.append(sent_index)

    global final_summary
    global visited_sentences
    #max_words_limit= int(len(keyvalue)*0.5)
    max_words_limit= 5
    threshold = 1
    final_summary = []
    visited_sentences = []
    summary_length = 0
    #print("1")
    #print(list_of_candidates_summeries)   
    candidate_summ = reorder(list_of_candidates_summeries)
    #print("2")
    #print(candidate_summ)
    
    candidate_summ_len = 0
    for index,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + 1
    candidate_summ = reorder(list_of_candidates_summeries)

    candidate_summ_len = 0
    for index,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + get_words_count(sent)


    while(True):
        if(candidate_summ_len > (max_words_limit + threshold)):
            if(candidate_summ_len != last_summ_len):
                Cand_edge,list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph_ara(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria) 

            elif(out_carteria != "max"):
                out_carteria="max"
                Cand_edge,list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph_ara(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria)
            else:
                list_of_candidates_summeries = candidate_summ
                break
        elif(candidate_summ_len < (max_words_limit)):
            candidate_summ = Last_summ
            list_of_candidates_summeries = candidate_summ
            break
        else:
            list_of_candidates_summeries = candidate_summ
            break
    
    n_clusters,cluster_map = post_processing_ar(list_of_candidates_summeries)
    final_summary = algorithm_4_ar(summary_length,threshold,max_words_limit,list_of_candidates_summeries,n_clusters,cluster_map)
    final = reorder(final_summary)
    outsumm = concat(final)
    
    return outsumm
    

## to run the code 
## summary = summarization_ara(your input text)

يحتاج الفرد بين فترة وأخرى إلى كسر الروتين والعادة اليومية لتجديد نشاطه، وإن أكثر ما يوقف الإنسان هي حالة الرتابة والثبات على عادة واحدة فقط.
كل شيء في الحياة يحتاج إلى التجديد والتغيير حتى نستشعر روح المغامرة وروح الابتكار، لذا بعد كل إجازة ينتظرنا شيء مملوء عزيمة، تتجدد معه النوايا، وتتجدد معه النظرة المتفائلة، ماذا سنقدم من جديد؟ وكأنها بداية صفحة جديدة، نخطط ونبتكر طريقة معينة للعطاء في العمل وتجديد النشاط وتنويع الأساليب التي تطور العمل.
الإجازة جميلة جداً، نغير فيها نمطية حياتنا ونرتاح، ونستمتع بالأشياء مغايرة من سفر وزيارات واسترخاء، وكل منا له طريقته الخاصة في استغلال إجازته، وهي في الحقيقة تكسر بين عادة وأخرى، وبين نمط وآخر، وبين الركود والنشاط، لذا تجدد الإجازة كل شيء فينا.
دائماً نسأل أنفسنا ماذا نقدم بعد الإجازة؟ وكيف نستقبل العمل ونحن في نشاط وحب؟ هذه الأسئلة إثراء لنا، لكن للأسف نلاحظ أن هناك من يشوه صورة العمل وكأن الحياة من المفترض بها أن تكون كلها إجازة، وهنا لا بد أن نتعمق في مفهوم العمل والإجازة، هناك من ينظر للعمل كواجب أو من أجل الراتب فقط، وهناك من يعيش حالة من الوعي العميقة في نظرته للحياة، وأن العمل جزء جميل من حياة الإنسان، نحن في العمل نفكر ونخطط ونجتهد ونبذل الطاقة والجهد في تقديم أحسن ما لدينا، وهذا مرتبط بالوعي لأهمية رسالتنا في الحياة، وأن كلاً منا له رسالة مهنية واجتماعية وعائلية، فمن يعي ذلك سيعيش مع رسالته في تناغم جميل وعجيب، لذا سيعرف ويدرك كيف يستفيد من أوقات إجازته في التجديد والراحة، ويدرك أيضاً أهمية العمل بالنسبة له، وأنه ليس واجباً فقط بل حياته، كيف يقضيها؟ وكيف يوازن بين ما يحبه ودوره ورسالته والقدرة على الاستمتاع في كل منهما والتي نطلق على القدرة كلمة التناغم؟.
التناغم يجعلنا ننتقل من حال لآخر، ومن مرحلة لأخرى، متقبلين لكل تغيير مؤقت أو دائم، هذا التغيير هو من يقضي على الرتابة والروتين، ويجعلنا في حالة دائمة مستغرقة في جمال الحياة، الشيء الوحيد الذي يقتل الإنسان هو حالة الجمود والثبات.
لذا كيف ستكون العودة إلى العمل؟ وبأي روح نستقبله؟ وبأي عزيمة وإصرار نتقبل يوماً جديداً ونحن أكثر بهجة وقوةً؟
من المهم ترك الصورة التقليدية للعمل والصورة المشوهة له، ونستبدلها بالصورة الجميلة، وأنها حياة ممتلئة بالنشاط والحركة والإنتاج.
العمل يحرك الفكر ويجدده، والعمل يحرك الجسد وينشطه، والعمل يجعلنا جميعاً نستشعر قيمة الأشياء التي نقوم بها، وأن لنا دوراً كبيراً في نهضة بلدنا ومجتمعنا والأكيد لأنفسنا أيضاً، كل ذلك يتطلب نفض تراب الكسل، وإدراك أهمية العمل بنظرة جديدة ومفهوم جديد.

نقلا عن الرياض

In [24]:
text = """يحتاج الفرد بين فترة وأخرى إلى كسر الروتين والعادة اليومية لتجديد نشاطه، وإن أكثر ما يوقف الإنسان هي حالة الرتابة والثبات على عادة واحدة فقط.
كل شيء في الحياة يحتاج إلى التجديد والتغيير حتى نستشعر روح المغامرة وروح الابتكار، لذا بعد كل إجازة ينتظرنا شيء مملوء عزيمة، تتجدد معه النوايا، وتتجدد معه النظرة المتفائلة، ماذا سنقدم من جديد؟ وكأنها بداية صفحة جديدة، نخطط ونبتكر طريقة معينة للعطاء في العمل وتجديد النشاط وتنويع الأساليب التي تطور العمل.
الإجازة جميلة جداً، نغير فيها نمطية حياتنا ونرتاح، ونستمتع بالأشياء مغايرة من سفر وزيارات واسترخاء، وكل منا له طريقته الخاصة في استغلال إجازته، وهي في الحقيقة تكسر بين عادة وأخرى، وبين نمط وآخر، وبين الركود والنشاط، لذا تجدد الإجازة كل شيء فينا.
دائماً نسأل أنفسنا ماذا نقدم بعد الإجازة؟ وكيف نستقبل العمل ونحن في نشاط وحب؟ هذه الأسئلة إثراء لنا، لكن للأسف نلاحظ أن هناك من يشوه صورة العمل وكأن الحياة من المفترض بها أن تكون كلها إجازة، وهنا لا بد أن نتعمق في مفهوم العمل والإجازة، هناك من ينظر للعمل كواجب أو من أجل الراتب فقط، وهناك من يعيش حالة من الوعي العميقة في نظرته للحياة، وأن العمل جزء جميل من حياة الإنسان، نحن في العمل نفكر ونخطط ونجتهد ونبذل الطاقة والجهد في تقديم أحسن ما لدينا، وهذا مرتبط بالوعي لأهمية رسالتنا في الحياة، وأن كلاً منا له رسالة مهنية واجتماعية وعائلية، فمن يعي ذلك سيعيش مع رسالته في تناغم جميل وعجيب، لذا سيعرف ويدرك كيف يستفيد من أوقات إجازته في التجديد والراحة، ويدرك أيضاً أهمية العمل بالنسبة له، وأنه ليس واجباً فقط بل حياته، كيف يقضيها؟ وكيف يوازن بين ما يحبه ودوره ورسالته والقدرة على الاستمتاع في كل منهما والتي نطلق على القدرة كلمة التناغم؟.
التناغم يجعلنا ننتقل من حال لآخر، ومن مرحلة لأخرى، متقبلين لكل تغيير مؤقت أو دائم، هذا التغيير هو من يقضي على الرتابة والروتين، ويجعلنا في حالة دائمة مستغرقة في جمال الحياة، الشيء الوحيد الذي يقتل الإنسان هو حالة الجمود والثبات.
لذا كيف ستكون العودة إلى العمل؟ وبأي روح نستقبله؟ وبأي عزيمة وإصرار نتقبل يوماً جديداً ونحن أكثر بهجة وقوةً؟
من المهم ترك الصورة التقليدية للعمل والصورة المشوهة له، ونستبدلها بالصورة الجميلة، وأنها حياة ممتلئة بالنشاط والحركة والإنتاج.
العمل يحرك الفكر ويجدده، والعمل يحرك الجسد وينشطه، والعمل يجعلنا جميعاً نستشعر قيمة الأشياء التي نقوم بها، وأن لنا دوراً كبيراً في نهضة بلدنا ومجتمعنا والأكيد لأنفسنا أيضاً، كل ذلك يتطلب نفض تراب الكسل، وإدراك أهمية العمل بنظرة جديدة ومفهوم جديد.

نقلا عن الرياض"""

In [25]:
summarization_ara(text)

Print key value sentence
{0: 'يحتاج الفرد بين فترة وأخرى إلى كسر الروتين والعادة اليومية لتجديد نشاطه،', 1: 'وإن أكثر ما يوقف الإنسان هي حالة الرتابة والثبات على عادة واحدة فقط', 2: 'كل شيء في الحياة يحتاج إلى التجديد والتغيير حتى نستشعر روح المغامرة وروح الابتكار،', 3: 'لذا بعد كل إجازة ينتظرنا شيء مملوء عزيمة،', 4: 'تتجدد معه النوايا،', 5: 'وتتجدد معه النظرة المتفائلة،', 6: 'ماذا سنقدم من جديد؟', 7: 'وكأنها بداية صفحة جديدة،', 8: 'نخطط ونبتكر طريقة معينة للعطاء في العمل وتجديد النشاط وتنويع الأساليب التي تطور العمل', 9: 'الإجازة جميلة جداً،', 10: 'نغير فيها نمطية حياتنا ونرتاح،', 11: 'ونستمتع بالأشياء مغايرة من سفر وزيارات واسترخاء،', 12: 'وكل منا له طريقته الخاصة في استغلال إجازته،', 13: 'وهي في الحقيقة تكسر بين عادة وأخرى،', 14: 'وبين نمط وآخر،', 15: 'وبين الركود والنشاط،', 16: 'لذا تجدد الإجازة كل شيء فينا', 17: 'دائماً نسأل أنفسنا ماذا نقدم بعد الإجازة؟', 18: 'وكيف نستقبل العمل ونحن في نشاط وحب؟', 19: 'هذه الأسئلة إثراء لنا،', 20: 'لكن للأسف نلاحظ أن هناك من يشوه صورة العمل وكأن 

'وإن أكثر ما يوقف الإنسان هي حالة الرتابة والثبات على عادة واحدة فقط كل شيء في الحياة يحتاج إلى التجديد والتغيير حتى نستشعر روح المغامرة وروح الابتكار، لذا بعد كل إجازة ينتظرنا شيء مملوء عزيمة، وكل منا له طريقته الخاصة في استغلال إجازته، لكن للأسف نلاحظ أن هناك من يشوه صورة العمل وكأن الحياة من المفترض بها أن تكون كلها إجازة،'